# 1. Motivation

## The dataset

The data used comes from two sources. One is from https://datausa.io/ and contains information about wage within the two states: California and New York. The other is downloaded from kaggle and contains many different files that will need cleaning. The link to the data kaggle and the different files can be seen below. 

https://www.kaggle.com/datasets/justinas/startup-investments?select=funds.csv

**acquisitions.csv:** information about startups that have been bought (price and time)

**degrees.csv:** the education backgrounds of individuals involved in the startup world.

**funding_rounds.csv:** information about startup funding rounds (amounts of founds, dates, founding, the founder)

**funds.csv:** data on the venture capital funds that make investments.

**investments.csv:** data on the various different investments made by venture capitalists.

**ipos.csv:** data on initial public offerings (stock name)

**milestones.csv:** events within the startup ecosystem. (Milestone description)

**objects.csv:** Main file containing base information. (Name, category, status)

**offices.csv:** information about startup company offices (latitude and longitude)

**people.csv:** information about individuals in the startup world.

**relationships.csv:** relationship data that links companies to individuals and their positions.

## Why investment data and our end goal

We wanted to explore the world of starups, and investigate if there is a recipe for success. We choose the above data as it includes information about the status, location and funding/investments in the startup. As the overall theme in the assignment is arround cities, we investigate whether the location of the startup has an impact on the startup succes. 

The goal for the end user is to get an insight into what influences the success of a startup in order to replicate the recipe.  

***Our Website:*** https://rococo-moxie-b83ceb.netlify.app

# 2. Basic stats


**Data cleaning and preprocessing**

As the dataset consists of a combination of different datasets, we carefully select which columns from each dataset are kept. Some data is not relevant to our analysis, so we exclude it. An example is the source description included in some of the datasets. The notebook will contain many different date frames, which will be different combinations of the files. This choice was made as on big dataset would require too much deletion of data to avoid redundancy. Instead, some files are used in more than one combination. For example offices, that is both combined with the relationships for educational background and funding for information based on location.   

**Dataset stats, key points from exploratory data analysis**

We choose only to look at US as it is the country with the most startups in our dataset. Some key findings in the analysis included that the majority of the data is on operating startups, making a comparison between success and failure more difficult. Instead, the exploratory analysis is more focused on the aspect that categorizes the operating startups. 

## 2.1 Package installation

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as pg
import numpy
import plotly.graph_objs as go
from folium import plugins
from plotly.subplots import make_subplots
import folium
import matplotlib as plt
import seaborn as sns
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from folium.plugins import HeatMap
import contractions
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from wordcloud import ImageColorGenerator 
from PIL import Image
from operator import itemgetter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.collocations import *
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.palettes import Category20b, YlGnBu
import nltk
from bokeh.models import Title
nltk.download('wordnet')
import stylecloud
import re
import matplotlib.pyplot as plt
from bokeh.plotting import figure
from bokeh.palettes import magma, viridis
from folium import plugins
from bokeh.models import ColumnDataSource, HoverTool, FactorRange, Legend
from bokeh.io import output_file, show, output_notebook, push_notebook

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/helenehjort/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
sti=r"/Users/helenehjort/Library/Mobile Documents/com~apple~CloudDocs/Business Analytics/8. Semester/02806 Social data analysis/Project/Data"
#sti=r"/Users/kathrinesofierasmussen/Library/CloudStorage/GoogleDrive-rasmussen.kathrine20@gmail.com/Mit drev/DTU/M.Sc./2. Semester/02806 Social Data Analysis and  Visualization /Assignments/ProjectData"
#sti=r"/Users/stj/Documents/DTU_BA/2_Semester/Socialdata/archive"

#Read the data
acquisitions = pd.read_csv(sti+"/acquisitions.csv")
degrees = pd.read_csv(sti+"/degrees.csv")
funding_rounds = pd.read_csv(sti+"/funding_rounds.csv")
founds = pd.read_csv(sti+"/funds.csv")
investments = pd.read_csv(sti+"/investments.csv")
ipos = pd.read_csv(sti+"/ipos.csv")
milestones = pd.read_csv(sti+"/milestones.csv")
objects = pd.read_csv(sti+"/objects.csv")
offices = pd.read_csv(sti+"/offices.csv")
people = pd.read_csv(sti+"/people.csv")
relationships = pd.read_csv(sti+"/relationships.csv")

/var/folders/xt/ymkjwn212xzf8xqtdvj5kffc0000gn/T/ipykernel_3711/3987843616.py:13: DtypeWarning:

Columns (3,7,9,10,17,18,21,22,23,25,26,29,30,37) have mixed types. Specify dtype option on import or set low_memory=False.



## 2.2 Offices, contry, city

The **Office** dataset holds information about the location of the offices for the different companies.  

Before desciding on columns to keep and which to discard, we do an initial analysis on NaN-values. We are certeintly interested in which country has the highest amount of startups, that is investigated now:

In [3]:
print("Number of countries in the dataset = ", offices['country_code'].nunique())
print("/n The different countries and their occurence= ", offices['country_code'].value_counts())

Number of countries in the dataset =  184
/n The different countries and their occurence=  USA    62991
GBR     9719
IND     5099
CAN     4574
DEU     2768
       ...  
IRQ        1
BEN        1
GIN        1
FJI        1
NCL        1
Name: country_code, Length: 184, dtype: int64


We see that USA is clearly overrepresented and therefore we choose to countinue with that. The visuaization for the reader will be made and elaborated later on. 

In [4]:
# We on want to keep data from usa
usa_offices=offices.loc[offices['country_code'] == 'USA']

We now check for null-values. Aboved showed a dataframe consisting of 62991 entities. Compared with below, we see that important entries missing values are `City` and `State_code`. Null-values within that field will be removed. Other columns such as `Description`, `address1`, `address2`, `zip_code`, `country_code`, `created_at`, `updated_at` will be removed, as it includes non-usefull information for this matter.

In [5]:
usa_offices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62991 entries, 0 to 112717
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            62991 non-null  int64  
 1   object_id     62991 non-null  object 
 2   office_id     62991 non-null  int64  
 3   description   35278 non-null  object 
 4   region        62991 non-null  object 
 5   address1      52775 non-null  object 
 6   address2      25148 non-null  object 
 7   city          61184 non-null  object 
 8   zip_code      55250 non-null  object 
 9   state_code    61722 non-null  object 
 10  country_code  62991 non-null  object 
 11  latitude      62991 non-null  float64
 12  longitude     62991 non-null  float64
 13  created_at    0 non-null      float64
 14  updated_at    0 non-null      float64
dtypes: float64(4), int64(2), object(9)
memory usage: 7.7+ MB


In [6]:
# Remove / exclude columns: description, address1, address2, zip_code, country_code, created_at, updated_at

# Columns to keep are the once listed below
usa_offices = usa_offices.loc[:, ['id', 'object_id','office_id','region', 'city', 'state_code', 'latitude', 'longitude']]

In [7]:
# Drop rows with NaN values in 
usa_offices.dropna(subset=['city', 'state_code'], how='any', inplace=True)

## 2.3 Main dataset about the company and its finances
The **Objects** dataset holds information about the status of the company for example if it has been acquired, the price of the acquired company is in **acquisitions**, we merge these to get the status and price in a table. **Ipos** contains data on initial public offerings which we merge with the others.

In [8]:
main_data = pd.merge(objects, acquisitions, left_on='id', right_on='acquired_object_id', how='left')
main_data = pd.merge(main_data, ipos, left_on='id_x', right_on='object_id', how='left')

In [9]:
# Only keep data from USA
main_data = main_data[main_data['id_x'].isin(usa_offices['object_id'])]

Below can be seen the choosen information to drop in each dataset. It can either due to redudant information 

In [10]:
# removed columns from objects: 'normalized_name', 'domain', 'homepage_url', 'twitter_username', 'logo_url', 'logo_width', 'logo_height', 'short_description', 'country_code',
# 'state_code', 'city', 'region', 'first_milestone_at', 'last_milestone_at', ,'parent_id', 'entity_id', 'first_funding_at', 'last_funding_at',

# removed columns from acquisitions: 'id_y', 'acquisition_id','acquiring_object_id', 'source_url', 'source_description', 'created_at_y', 'updated_at_y'

# removed columns from ipos: 'id', 'object_id', 'source_url_y', 'source_description_y', 'created_at_x', 'updated_at_x',


# columns kept 
main_data = main_data.loc[:, ['id_x', 'entity_type','entity_id', 'name', 'category_code', 'status', 'founded_at', 
                              'closed_at', 'description','overview', 'tag_list','state_code', 'first_investment_at', 'last_investment_at', 'investment_rounds',
       'invested_companies', 'funding_rounds', 'funding_total_usd',  'milestones', 'relationships', 'created_by',
       'created_at_x', 'updated_at_x',  'acquired_object_id', 'term_code',
       'price_amount', 'price_currency_code', 'acquired_at', 'ipo_id','valuation_amount', 'valuation_currency_code',
       'raised_amount', 'raised_currency_code', 'public_at', 'stock_symbol']]

After excluding the non-important variables, we take a look into the remaining and their null-values. As above, `State_code` is an important variable, and will be troughout, der fjerenes derfor null-values within that category, together with the `category_code` as it includes important information about the industry within the startip. This is a variable we think has a influence on location and partly the succes of the startup and therefore null-values are not tolerated.

In [11]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55463 entries, 0 to 208355
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_x                     55463 non-null  object 
 1   entity_type              55463 non-null  object 
 2   entity_id                55463 non-null  int64  
 3   name                     55463 non-null  object 
 4   category_code            47169 non-null  object 
 5   status                   55463 non-null  object 
 6   founded_at               41244 non-null  object 
 7   closed_at                1322 non-null   object 
 8   description              35148 non-null  object 
 9   overview                 53795 non-null  object 
 10  tag_list                 29828 non-null  object 
 11  state_code               53739 non-null  object 
 12  first_investment_at      3896 non-null   object 
 13  last_investment_at       3896 non-null   object 
 14  investment_rounds    

In [12]:
# Drop rows with NaN values in 
main_data.dropna(subset=['category_code', 'state_code'], how='any', inplace=True)

We want to delimit the area of analysis more and we therefore look down to which states provide the basis for the most startups. This is done already in the 'cleaning' of the data, as hopefully a pattern is seen and in that case will be consistent for the rest of the analysis. 

In [13]:
print("The different states and their number of occurence= ", main_data['state_code'].value_counts()[:10])

The different states and their number of occurence=  CA    15006
NY     5201
MA     2675
TX     2516
FL     1933
WA     1702
IL     1555
PA     1170
CO     1066
NJ     1051
Name: state_code, dtype: int64


We see a big jump between CA and NY and once again from NY down to MA. However, in order to identify a general pattern and also differentiate between country areas, it is decided to proceed with both CA and NY.  As they have different locations it is therefore also possible to differentiate between their local circumstances. Even if we choose to look only at CA and NY, an `Others` category is created for all other states. This is done so that strong trends in other states are not overlooked. However, these are not examined further at the state and city level. 

In [14]:
# Create a list of conditions 
conditions = [(main_data['state_code'] == "CA"),
            (main_data['state_code'] == "NY"),
            (main_data['state_code'] != "CA") & (main_data['state_code'] != "NY")]

# Create a list of values for conditions

values = ["CA", "NY", "Others"]

main_data["State_Group"] =  np.select(conditions, values)

## 2.4 Founder

The datasets **people** and **degrees** contain data about the founder of the company, we use **relationship** to be able to merge the person with the company. 

In [15]:
#We merge the datasets
founders = pd.merge(relationships, people, left_on='person_object_id', right_on='object_id', how='left')
founders = pd.merge(founders, degrees, left_on='person_object_id', right_on='object_id', how='left')
founders = pd.merge(founders, offices, left_on='relationship_object_id', right_on='object_id', how='left') 

/var/folders/xt/ymkjwn212xzf8xqtdvj5kffc0000gn/T/ipykernel_3711/124753642.py:4: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'id_x'} in the result is deprecated and will raise a MergeError in a future version.



In [16]:
founders.columns

Index(['id_x', 'relationship_id', 'person_object_id', 'relationship_object_id',
       'start_at', 'end_at', 'is_past', 'sequence', 'title', 'created_at_x',
       'updated_at_x', 'id_y', 'object_id_x', 'first_name', 'last_name',
       'birthplace', 'affiliation_name', 'id_x', 'object_id_y', 'degree_type',
       'subject', 'institution', 'graduated_at', 'created_at_y',
       'updated_at_y', 'id_y', 'object_id', 'office_id', 'description',
       'region', 'address1', 'address2', 'city', 'zip_code', 'state_code',
       'country_code', 'latitude', 'longitude', 'created_at', 'updated_at'],
      dtype='object')

In [17]:
# removed columns from relationships: 'start_at', 'end_at', 'is_past', 'sequence', 'created_at', 'updated_at'

# removed columns from degrees: 'created_at', 'updated_at'

# removed columns from offices: 'adress1', 'adress2', 'source_url_y', 'zip_code', 'created_at', 'updated_at',


# columns kept 
founders = founders.loc[:, ['id_x', 'relationship_id','person_object_id', 'relationship_object_id', 'title', 'id_y', 'object_id_x', 
                              'first_name', 'last_name','birthplace', 'affiliation_name','id_x', 'object_id_y', 'degree_type', 'subject',
       'institution', 'graduated_at', 'id_y',  'object_id', 'office_id', 'description',
       'region', 'city',  'state_code', 'country_code',
       'latitude', 'longitude']]

In [18]:
# We only keep data from USA
founders = founders[founders['country_code']=="USA"]

It can be seen that there exicsts a small bias in the data, as the number of unique id's and object id's under the degree table is not the same. After examining the data it is found out that it is due to one person being registrered for all their levels of degree and sometimes even more subjects within their bachelors degree. In order for us to get a correct view of the locations of the founders education, we have choosen to keep distinct rows broken down by `degree_type` and `institution`. 

This means that persons with different levels of education from the same school are saved, but should it happen that both a minor and a major are registered or that the person has taken two courses of the same degree from the same school, this is not counted. Please note that this is a subjective way of telling the story, but it is the sorting that will give us the most objective data for telling the story.

Below we only look at the data frame 'degree', for the sake of clarity of the study. It contains the persons' id 'object_id' and therefore you still get a true picture

In [19]:
#We check the quality of the merged data 
degrees.nunique()

id              109610
object_id        68451
degree_type       7148
subject          20050
institution      21067
graduated_at        71
created_at       81162
updated_at       80873
dtype: int64

In addition one company could have offices in more places, so an extra element to look at is the state_code. We thereby also drop everythin but the last row, with same state_code. For now the dataframe could consists of duplicate founders if the company have offices located in more states and/or countrys, but that will automatically be removed, during further sorting of the dataframe such as only looking at the states 'NY' and 'CA'. 

In [20]:
# Data cleaning 
founders = founders.drop_duplicates(
  subset = ['person_object_id', 'degree_type', 'institution', 'state_code'],
  keep = 'last').reset_index(drop = True)

In [21]:
founders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271897 entries, 0 to 271896
Data columns (total 31 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id_x                    271897 non-null  int64  
 1   id_x                    171885 non-null  float64
 2   relationship_id         271897 non-null  int64  
 3   person_object_id        271897 non-null  object 
 4   relationship_object_id  271897 non-null  object 
 5   title                   264028 non-null  object 
 6   id_y                    271692 non-null  float64
 7   id_y                    271897 non-null  float64
 8   object_id_x             271692 non-null  object 
 9   first_name              271692 non-null  object 
 10  last_name               271692 non-null  object 
 11  birthplace              25272 non-null   object 
 12  affiliation_name        271652 non-null  object 
 13  id_x                    271897 non-null  int64  
 14  id_x                

As we want to investigate the degree of the founder and the employees, we do not tolerate null-values within `degree_type`, they will therefore be removed.

In [22]:
#We want to investigate whether educational background has an influence on start-up success, therefore all empty rows are removed.
founders.dropna(subset=['degree_type'], inplace=True)

In [23]:
# We extract the year the employees graduated in
founders['graduated_at'] = pd.to_datetime(founders['graduated_at'], format = '%Y-%m-%d')
founders['grad_year'] = founders['graduated_at'].dt.year

Earlier investigation of the states shows us the two largest states: CA and NY, we know make groups so we can compare them to the rest of the USA

In [24]:
# Create a list of conditions 
conditions = [(founders['state_code'] == "CA"),
            (founders['state_code'] == "NY"),
            (founders['state_code'] != "CA") & (founders['state_code'] != "NY")]

# Create a list of values for conditions
values = ["CA", "NY", "Others"]

founders["State_Group"] =  np.select(conditions, values)

## 2.5 Founding and investments

The datasets **founds**, **investments**, and **founding_round** contain data about the funding and investment made in startups. We want the data on the funding and the funding round in the same data frame, so we use the funded_object_id to merge. That data frame is then merged with offices to identify where in the country the startups that have received funding are located. 

In [25]:
offices

,id,object_id,office_id,description,region,address1,address2,city,zip_code,state_code,country_code,latitude,longitude,created_at,updated_at
0,1,c:1,1,NaN,Seattle,710 - 2nd Avenue,Suite 1100,Seattle,98104,WA,USA,47.603122,-122.333253,NaN,NaN
1,2,c:3,3,Headquarters,SF Bay,4900 Hopyard Rd,Suite 310,Pleasanton,94588,CA,USA,37.692934,-121.904945,NaN,NaN
2,3,c:4,4,NaN,SF Bay,135 Mississippi St,NaN,San Francisco,94107,CA,USA,37.764726,-122.394523,NaN,NaN
3,4,c:5,5,Headquarters,SF Bay,1601 Willow Road,NaN,Menlo Park,94025,CA,USA,37.416050,-122.151801,NaN,NaN
4,5,c:7,7,NaN,SF Bay,Suite 200,654 High Street,Palo Alto,94301,CA,ISR,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112713,112714,f:15098,127846,NaN,New York,8 Sound Shore Drive,Suite 303,Greenwich,06830,CT,USA,0.000000,0.000000,NaN,NaN
112714,112715,c:286200,127847,NaN,Santa Barbara,735 State Street,Suite 500,Santa Barbara,93101,CA,USA,0.000000,0.000000,NaN,NaN
112715,112716,c:256895,127848,NaN,Los Angeles,"5777 W. Century Blvd., Suite 360",NaN,Los Angeles,90045,CA,USA,0.000000,0.000000,NaN,NaN
112716,112717,c:256200,127849,NaN,New York,420 LExington Avenue,NaN,New York,10170,NY,USA,0.000000,0.000000,NaN,NaN


In [26]:
#The data is merged
funding_data = pd.merge(investments[['funded_object_id', 'investor_object_id']] ,funding_rounds[['object_id', 'funded_at', 'funding_round_type', 'raised_amount_usd', 'pre_money_valuation_usd', 'post_money_valuation_usd', 'participants', 'is_first_round', 'is_last_round']], left_on='funded_object_id', right_on='object_id', how='left')
funding_data = pd.merge(funding_data, offices[['object_id', 'office_id', 'description', 'region', 'city', 'state_code', 'country_code', 'latitude', 'longitude']], left_on='funded_object_id', right_on='object_id', how='left')

In [27]:
# Following colums are removed:

# investments: funding_round_id, created_at, updated_at

# funding_rounds: funding_round_id, funding_round_code, raised_amount, raised_currency_code, pre_money_valuation,
# post_money_valuation, source_url, source_description, created_by, created_at, updated_at

# offices: address1, address2, zip_code, created_at, updated_at

In [28]:
#We only want to look at the USA
funding_data = funding_data[funding_data['country_code'] == "USA"]

In [29]:
# Investigating nul-values
funding_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214585 entries, 0 to 297341
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   funded_object_id          214585 non-null  object 
 1   investor_object_id        214585 non-null  object 
 2   object_id_x               214585 non-null  object 
 3   funded_at                 214436 non-null  object 
 4   funding_round_type        214585 non-null  object 
 5   raised_amount_usd         214585 non-null  float64
 6   pre_money_valuation_usd   214585 non-null  float64
 7   post_money_valuation_usd  214585 non-null  float64
 8   participants              214585 non-null  int64  
 9   is_first_round            214585 non-null  int64  
 10  is_last_round             214585 non-null  int64  
 11  object_id_y               214585 non-null  object 
 12  office_id                 214585 non-null  float64
 13  description               120549 non-null  o

Again we don't tolerate null-values within `state_code`, these will be removed. 

In [30]:
funding_data.dropna(subset=['state_code'], inplace=True)

In [31]:
#The date and time values are transformed
funding_data['funded_at']= pd.to_datetime(funding_data['funded_at'], format='%Y-%m-%d')
funding_data['Weekday'] = funding_data['funded_at'].dt.day_name()
funding_data['Month'] = funding_data['funded_at'].dt.month_name()

In [32]:
#The funding data is grouped into three. The states CA, NY and all the rest. 

# Create a list of conditions 
conditions = [(funding_data['state_code'] == "CA"),
            (funding_data['state_code'] == "NY"),
            (funding_data['state_code'] != "CA") & (funding_data['state_code'] != "NY")]

# Create a list of values for conditions

values = ["CA", "NY", "Others"]

funding_data["State_Group"] =  np.select(conditions, values)

In [33]:
# The object data set is also grouped into three for later use.
# Create a list of conditions 
conditions = [(objects['state_code'] == "CA"),
            (objects['state_code'] == "NY"),
            (objects['state_code'] != "CA") & (objects['state_code'] != "NY")]

# Create a list of values for conditions

values = ["CA", "NY", "Others"]

objects["State_Group"] =  np.select(conditions, values)

# 3. Data Analysis & Visualizations (5)

The data analysis and visualisations made on that behalf, follows the structure of the article and are therefore tightly connected. Therefore, it is chosen to show them together, with the explanation of the visualization and why it is interesting for the analysis. 

## 3.1 Introduction

In order to make the best possible analysis in which correlations can be identified, we first look at which country contains the most start-ups. This is done in order to have the same cultural and structural basis for identifying patterns and variables that influence the success of a startup.  

In [34]:
# distribution of countries
county_counts = offices['country_code'].value_counts()
df_county_counts = pd.DataFrame({'country_code': county_counts.index, 'count': county_counts.values})
df_county_counts =df_county_counts[0:10]

# plot
fig1 = px.bar(df_county_counts, x='country_code', y='count')
fig1.update_traces(marker_color=px.colors.sequential.deep[::-1], width=0.8)


# Get data for the first plot
state_counts = offices['country_code'].value_counts().reset_index()
state_counts.columns = ['country_code', 'count']

# Create choropleth plot for the first plot
fig2 = go.Figure(go.Choropleth(
    locations=state_counts['country_code'],
    z=state_counts['count'],
    locationmode='ISO-3',
    colorscale='YlGnBu',
    zmin=0,
    zmax=state_counts['count'].max(),
    marker_line_width=0
))

# Create subplot
fig = make_subplots(rows=1, cols=2, 
                    specs=[[{'type': 'bar'}, {'type': 'choropleth'}]],
                    column_widths=[0.3, 0.7]) # adjust column widths
fig.add_trace(fig1.data[0], row=1, col=1)
fig.add_trace(fig2.data[0], row=1, col=2)
fig.update_layout( title_text='Top 10 Largest Startups Countries and Number of Startups Worldwide',font=dict(size=12),
                  #plot_bgcolor='white', 
                  font_family='Arial',
                  title_x=0.5
                  )
fig.show()
pyo.plot(fig, filename='plot/country.html')

'plot/country.html'

It is seen that the US counts for the greatest share of startups represented in our data. A continuous sorting of countries is made in previous sections, as this view should be valid throughout the rest of the analysis. The visualisation is choosen to be included in the article as it creates a great understanding why the US is the point of view. 

As the data is spread out over a long period of time, we are now looking to see if we can identify any trends that needs to be taken into account during the analysis. For instance, it could be expected that there were fewer startups in and around 2007 due to the financial crisis. 

In [35]:
# subsection of data that counts the number of times there has been registrered a startup
main_data['founded_at'] = pd.to_datetime(main_data['founded_at'])
df_year_count = main_data.groupby(main_data['founded_at'].dt.year).count()

# date filter
df_year_count = df_year_count[df_year_count.index >= 1990] 
df_year_count = df_year_count[df_year_count.index <2014]

# Plot figure over counts
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_year_count.index, y=df_year_count['founded_at'], line=dict(color='#2B6168')))
fig.update_layout(title='Number of Companies Founded Each Year in US', 
                  xaxis_title='Year', 
                  yaxis_title='Amount of startups',
                  font=dict(size=12),
                  plot_bgcolor='white', 
                  font_family='Arial',
                  title_x=0.5)
fig.show()
pyo.plot(fig, filename='plot/year_count.html')

'plot/year_count.html'

Expectations did not materialize, on the contrary, we are seeing an increase in the number of start-ups following the financial crisis. 

A basic idea behind the project is to compare sold startups with still running startups in order to identify the reasons behind the sale, as a sale of a startup is considered a success. The next plot illustrates the distribution of the operating status of the different startups. 

Normally, it is not recommended to use pie charts for visualizations of distributions. However, because there are only four categories and three of them account for such a small proportion, the pie chart helps to emphasize to the reader how large a proportion is still in operation. This helps to support the truth value to the reader. 

In [36]:
# Count on the number of start-ups under each status category
status_counts = main_data['status'].value_counts()
df_status_counts = pd.DataFrame({'status': status_counts.index, 'count': status_counts.values})

# Plot the count
fig = px.pie(df_status_counts, values='count', names='status')
fig.update_traces(textposition='inside', textinfo='percent+label', marker=dict(colors=px.colors.sequential.YlGnBu[::-1]))
fig.update_layout( title= 'Status of Startups Companies in US',
                  font=dict(size=12),
                  plot_bgcolor='white', 
                  font_family='Arial',
                  title_x=0.5
)

fig.show() 
pyo.plot(fig, filename='plot/circle.html')

'plot/circle.html'

As can be seen from the plot, almost 90% are still in operation and therefore it does not make sense to compare, as the amount of data for closed/acquired companies may constitute a bias for the final conclusion. Therefore, we drop this perspective and examine more closely the general trends observed in the United States. 

## 3.2 General trends across US

We have some ideas of our own about which cities and states could act as hubs for startups, but to hold the reader's hand and to confirm our hypothesis, a plot on the amount of startups in each state is in order. This is also done so that we can once again limit ourselves to one or more areas so that we can more accurately explore the cities and states that have the most potential for startups. 

In [37]:
# Count of number of startups in each state
state_counts = usa_offices['state_code'].value_counts().reset_index()

# only include state code and their count. 
state_counts.columns = ['state_code', 'count']

# plot figure
fig = px.choropleth(state_counts, locations='state_code', color='count',
                    locationmode='USA-states', scope='usa',
                    color_continuous_scale='YlGnBu',
                    range_color=(0, state_counts['count'].max()),
                    labels={'count': 'Number of startups'})

# Extra to layout, such as title and funts 
fig.update_layout(title_text='Number of Startups by State', font=dict(size=12),
                  plot_bgcolor='white', 
                  font_family='Arial',
                  title_x=0.5)
fig.show()
pyo.plot(fig, filename='plot/cities.html')

'plot/cities.html'

We see that California and New York clearly have the most startups, which is to be expected as the two states contains large cities. This visual is choosen to be included in the story, as it clearly shows the existence of startup hubs in the US. The plot is also meant to be a basic plot that further helps to introduce the reader to the story. 

As the two states are geographically far apart, it is now investigated whether there is a difference in the type of startups in the two states. This is interesting in order to see if the two states differs within the startup industries. 

In [38]:
# count of top 9 industries within all data
Top9_category = main_data[main_data["category_code"].isin(list(main_data["category_code"].value_counts().head(9).index))]

# create dataframe for plot, groupby the industry and the states
category_code = Top9_category.groupby(['category_code', 'State_Group']).size().unstack('State_Group').fillna(0)

# reset index of datafram
category_code = category_code.reset_index().rename_axis(columns={'': ''})

# name output html-file for the website
output_file("plot/CategoryStartups.html")

# State groups
States = set(["NY", "CA", "Others"])

# Create list of state categories
cat = list(category_code['category_code'])
cats = [str(c) for c in cat]

# Define colors
colors = YlGnBu[len(States)]

# Make Bokeh plot
source = ColumnDataSource(category_code)
p = figure(x_range=FactorRange(factors=cats), x_axis_label="Startup Categories", y_axis_label="Number of startups",min_width=900,min_height=300) 

bar ={} 
items = [] 
for indx,i in enumerate(States):
    bar[i] = p.vbar(x='category_code',  top=i, source= source, 
                 muted_alpha=0.5, muted=False, color = colors[indx])
    items.append((i, [bar[i]])) 

legend = Legend(items=items, location='top_right') 
p.add_layout(legend, 'right') 

p.legend.click_policy="mute"

p.title = Title(text='Categories of Startup Companies in US', align='center', text_font_size='12pt')

p.xaxis.axis_label_text_font_size = '12pt'
p.yaxis.axis_label_text_font_size = '12pt'
p.legend.label_text_font_size = '12pt'

show(p)

It can be seen that California has a high concentration of tech, biotech and enterprise startups. On the other hand, there are few companies in these fields in New York, but there is a higher concentration of startups in ecommerce and advertising. This supports the existence of a difference between the state and the type of startups. 

In addition to the fact that the visualization is interactive and the user can investigate the state grouping, it helps to tell that there are different types of startups represented in the different states. Exactly the last reason is paramount in the rationale for including this visualization, as it helps to answer and provide an insight into what makes and matters for a startup's success - namely its surrounding competitors and sparring partners. 


We have now identified which states have the most startups and which industries they are in. In doing so, we accepted the outcome of states because of the cities located in the respective states. Therefore, we are now investigating which specific cities have the most startups, in order to be able to put it in relation to our article. 

In [39]:
# count of startups in cities
city_counts = usa_offices['city'].value_counts()
df_city_counts = pd.DataFrame({'city': city_counts.index, 'count': city_counts.values})

# Only include top 9
df_city_counts =df_city_counts[0:9]


fig = px.bar(df_city_counts, x='city', y='count')
fig.update_layout(
    title='Top Cities in US with Most Startups Companies',
                font=dict(size=12),
                xaxis_title='Cities',
                yaxis_title='Number of startups', 
                plot_bgcolor='white', 
                font_family='Arial',
                title_x=0.5
  
)
fig.update_traces(marker_color=px.colors.sequential.YlGnBu[::-1], width=0.8)

fig.show()
pyo.plot(fig, filename='plot/cities2.html')

'plot/cities2.html'

Unsurprisingly, New York and San Francisco are seen as the top startup hubs. This is not surprising since in SF we have Silicon Valley and in NYC we have the Financial district in Downtown Manhattan. The visualization is not directly included, but is used to support our point about cities and location for the reader. 

We now take a closer look at the two states and their cities to see if we can find correlations between the cities and the startup industry. 

In [40]:
# Following are made for California
# merge main_data and usa_offices dataframe
CA_offices = pd.merge(usa_offices, main_data[['id_x', 'category_code','name']], left_on='object_id', right_on='id_x', how='left')

# Geographical bounds for visual, that covers the state of California
ca_bounds = {
    'lat_min': 32.5,
    'lat_max': 42.5,
    'lon_min': -124.5,
    'lon_max': -114.5
}
out_of_bounds = ~((CA_offices['latitude'] >= ca_bounds['lat_min']) & (CA_offices['latitude'] <= ca_bounds['lat_max']) &
                  (CA_offices['longitude'] >= ca_bounds['lon_min']) & (CA_offices['longitude'] <= ca_bounds['lon_max']))
out_of_bounds_indices = CA_offices.index[out_of_bounds].tolist()

# Drop the rows outside the CA bounds from the DataFrame
CA_offices = CA_offices.drop(index=out_of_bounds_indices)

# count the number of each startup industry
code_counts = CA_offices.groupby('category_code').size().reset_index(name='count')

# only look at top 9 industries
top_codes = code_counts.sort_values(by='count', ascending=False)['category_code'][:9]

# Define colorcodes for the different industries
CA_offices['color_code'] = CA_offices['category_code'].apply(lambda x: x.strip() if isinstance(x, str) and x.strip() in top_codes.values else 'other')

# create the scattermap
fig = px.scatter_mapbox(CA_offices, lat='latitude', lon='longitude', zoom=5,
                        center={'lat': 35.7783, 'lon': -119.4179},
                        mapbox_style='open-street-map', 
                        color='color_code',hover_name='name',
                        color_discrete_sequence=px.colors.sequential.YlGnBu_r,labels={'color_code': 'Type of Startup'})
fig.show()
pyo.plot(fig, filename='plot/CA.html')

'plot/CA.html'

The map shows what is already suspected, that the startups in SA I are dominated by software. It is also dominated by biotech, which was more surpricing. The visualization is chosen because it provides a good overview and the reader the opportunity to delve into whatever topic they find interesting. For example, there may be a reader with a web start-up, who finds it interesting to look into the locations of these. 

In [41]:
# Following are made for New York
# merge main_data and usa_offices dataframe
NY_offices = pd.merge(usa_offices, main_data[['id_x', 'category_code','name']], left_on='object_id', right_on='id_x', how='left')

# Geographical bounds for visual, that covers the state of New York
ny_bounds = {
'lat_min': 40.477399,
'lat_max': 45.01585,
'lon_min': -79.76259,
'lon_max': -71.75174
}
out_of_bounds = ~((NY_offices['latitude'] >= ny_bounds['lat_min']) & (NY_offices['latitude'] <= ny_bounds['lat_max']) &
                  (NY_offices['longitude'] >= ny_bounds['lon_min']) & (NY_offices['longitude'] <= ny_bounds['lon_max']))
out_of_bounds_indices = NY_offices.index[out_of_bounds].tolist()

# Drop the rows outside the CA bounds from the DataFrame
NY_offices = NY_offices.drop(index=out_of_bounds_indices)

# count the number of each startup industry
code_counts = NY_offices.groupby('category_code').size().reset_index(name='count')

# Only look at top 9 industries within NY
top_codes = code_counts.sort_values(by='count', ascending=False)['category_code'][:9]

# Define colorcodes for the different industries
NY_offices['color_code'] = NY_offices['category_code'].apply(lambda x: x.strip() if isinstance(x, str) and x.strip() in top_codes.values else 'other')

# create the scattermap
fig = px.scatter_mapbox(CA_offices, lat='latitude', lon='longitude', zoom=5,
                        center={'lat': 35.7783, 'lon': -119.4179},
                        mapbox_style='open-street-map', 
                        color='color_code',hover_name='name',
                        color_discrete_sequence=px.colors.sequential.YlGnBu_r,labels={'color_code': 'Type of Startup'})
fig.show()
pyo.plot(fig, filename='plot/NY.html')

'plot/NY.html'

The map of New York City confirms the knowledge that the city is dominated by a startup in the field of advertising and e-commerce. 

## 3.3 The need for funding

We know that it takes money to get a startup up and running and that getting it can be hard. However, it is essential to survive the first years with high costs but low income. Therefore we investigate where the startups get funding and if there is any pattern that can provide some advice for other startups. 

We want to prepare data to be used in the map. For this purpose, empty rows must be removed as well as rows with errors in longitude and latitude.

In [42]:
map_data=usa_offices
map_data= map_data[~(map_data['latitude'] == 0.0)]
map_data= map_data[~(map_data['longitude'] == 0.0)]

# remove latitude and logitude outside USA
us_bounds = {
    'lat_min': 24.5,
    'lat_max': 49.5,
    'lon_min': -124.5,
    'lon_max': -66.5
}
out_of_bounds = ~((map_data['latitude'] >= us_bounds['lat_min']) & (map_data['latitude'] <= us_bounds['lat_max']) &
                  (map_data['longitude'] >= us_bounds['lon_min']) & (map_data['longitude'] <= us_bounds['lon_max']))
out_of_bounds_indices = map_data.index[out_of_bounds].tolist()

# Drop the rows outside the USA bounds from the DataFrame
map_data = map_data.drop(index=out_of_bounds_indices)

# merge map_data and main_data dataframes and only keep the columns in main_data seen below
map_data = pd.merge(map_data, main_data[['id_x','founded_at', 'funding_total_usd', 'name']], left_on='object_id', right_on='id_x', how='left')

# remove all fundings which are 0
map_data= map_data[~(map_data['funding_total_usd'] == 0.0)]

# drope all nan values
map_data.dropna(subset=['funding_total_usd'], inplace=True)

# convert founding date to datetime and create a value count for each year, that can be used for the heatmap
map_data['founding_date'] =  pd.to_datetime(map_data['founded_at'])
map_data['founding_date'].dt.year.value_counts()

2007.0    594
2012.0    573
2011.0    572
2006.0    530
2008.0    461
2005.0    414
2010.0    388
2009.0    307
2004.0    298
2003.0    221
2000.0    204
2002.0    186
1999.0    181
2001.0    141
2013.0    126
1998.0     90
1997.0     57
1996.0     49
1995.0     36
1994.0     34
1992.0     27
1993.0     15
1986.0     14
1990.0     13
1987.0     10
1985.0     10
1982.0      7
1989.0      7
1991.0      6
1988.0      5
1980.0      4
1983.0      4
1984.0      4
1981.0      3
1976.0      3
1978.0      3
1979.0      3
1963.0      3
1968.0      3
1975.0      2
1954.0      2
1961.0      2
1948.0      2
1930.0      1
1906.0      1
1953.0      1
1919.0      1
1973.0      1
1917.0      1
1969.0      1
1947.0      1
1952.0      1
1944.0      1
1971.0      1
1945.0      1
1922.0      1
1937.0      1
Name: founding_date, dtype: int64

In [43]:
# Define location, mapstyle and zoom.
map_hooray = folium.Map(location=[37.0902, -95.7129],
                        tiles="cartodbpositron",
                        zoom_start=4,
                        width='100%', 
                        height='100%') 

# Specify which columns to use for coordinates in dataframe
heat_df = map_data[['latitude', 'longitude']]

# make order for years 
years = [1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013 ]

# Create weight column, using date
heat_df['Weight'] = map_data['founding_date'].dt.year
heat_df = heat_df[heat_df["Weight"].isin(years)]

# convert weigth (year) to float values, at drop rows if either lat, long or weight is 0. Then sort by year (weight) and sort then in cronological order, starting from 1998. 
heat_df['Weight'] = heat_df['Weight'].astype(float)
heat_df = heat_df.dropna(axis=0, subset=['latitude','longitude', 'Weight'])
heat_df.sort_values(by='Weight', ascending = True, inplace = True)


# List comprehension to make out list of lists
heat_data = [[[row['latitude'],row['longitude']] for index, row in heat_df[heat_df['Weight'] == i].iterrows()] for i in heat_df['Weight'].unique()]


hm = plugins.HeatMapWithTime(heat_data, gradient={0: '#FCE22A', 0.2: '#00FFCA', 0.4: '#8BF5FA', 0.6: '#3F979B', 1: '#088395'}, index = years, auto_play=True, max_opacity=0.8)
hm.add_to(map_hooray)

# Display the map
map_hooray
map_hooray.save('plot/heatmap.html')

/var/folders/xt/ymkjwn212xzf8xqtdvj5kffc0000gn/T/ipykernel_3711/720004712.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/xt/ymkjwn212xzf8xqtdvj5kffc0000gn/T/ipykernel_3711/720004712.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



The map above shows a distribution of funding that coalign with the placement of startups and supports the theory that placing your startup in big cities will heighten the chances of success. The map is interactive and shows the evolvement over time, providing the reader with easily comparable data and access to time trends. The trend shows some difference in the middle states, but overall they have always been a lot of funding in the costal states. Now we want to investigate if it is only the location that influences the funding or if outside factors such as average wage also have an impact. We suspect that is does and there we use the data 'workforce' from our other dataset  'DATA USA'. https://datausa.io

In [44]:
#Workforce = pd.read_csv("/Users/stj/Documents/DTU_BA/2_Semester/Socialdata/Data2/Workforce.csv")
Workforce = pd.read_csv("/Users/helenehjort/Library/Mobile Documents/com~apple~CloudDocs/Business Analytics/8. Semester/02806 Social data analysis/Website/Socialdata/Workforce.csv")
#Workforce = pd.read_csv("/Users/kathrinesofierasmussen/Library/CloudStorage/GoogleDrive-rasmussen.kathrine20@gmail.com/Mit drev/DTU/M.Sc./2. Semester/02806 Social Data Analysis and  Visualization /Assignments/ProjectData/Workforce.csv")

In [45]:
# All state codes, so we can make a new column for the heat map. 
state_code = {'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
              'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
              'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
              'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
              'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
              'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
              'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
              'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
              'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
              'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
              'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
              'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
              'Wisconsin': 'WI', 'Wyoming': 'WY', 'Puerto Rico': 'PR'}

Workforce['StateCode'] = Workforce['State'].map(state_code)


In [46]:
# create the heatmap
fig = px.choropleth(Workforce, locations='StateCode', color='Average Wage',
                    locationmode='USA-states', scope='usa',
                    color_continuous_scale='YlGnBu',
                    range_color=(0, Workforce['Average Wage'].max()),
                    labels={'Average Wage': 'Average Wage'})

# update layout, so title and other fonts are the same as other visuals
fig.update_layout(title_text='Average Wage by State', font=dict(size=12),
                  plot_bgcolor='white', 
                  font_family='Arial',
                  title_x=0.5)
fig.show()
pyo.plot(fig, filename='plot/wage.html')

'plot/wage.html'

The heatmap areas with high average wages are also the areas with a lot of funding, supporting the theory that is not only big cities that impact startup success but also the city characteristics such as average wage. Now we want to look more into the details of investments made. 

Below a time series plot is created over the aggerated funding throughout the data. We also considered a time series over the average, but we wanted to be able to investigate specific spikes, to identify if there was any learning to take from their success. 

In [47]:
funding_data['funded_at'].dt.year.value_counts()

2012.0    34433
2011.0    33671
2013.0    31965
2010.0    26862
2009.0    20732
2008.0    19098
2007.0    17505
2006.0    13073
2005.0     9027
2004.0     2259
2003.0     1138
2000.0      917
1999.0      859
2002.0      841
2001.0      603
1998.0       95
1960.0       21
1997.0        9
1995.0        9
1996.0        6
1987.0        1
Name: funded_at, dtype: int64

We see from above that the number of fundings below 2006, is relatively low. We therefore only look at 2006 and onwards in the following plot.

In [48]:
# We aggregate the funding data

# Prep af funding data 
funding_data = funding_data[~(funding_data['funded_at'] < '2006-01-01')]
funding_data_count = funding_data.groupby(['State_Group', 'funded_at']).size().unstack('State_Group').fillna(0) 
funding_data_count = funding_data_count.reset_index().rename_axis(columns={'funded_at': ''})

# Aggregation
funding_data_agg = funding_data.groupby(['State_Group', 'funded_at'])['raised_amount_usd'].sum().unstack('State_Group').fillna(0) 
funding_data_agg = funding_data_agg.reset_index().rename_axis(columns={'funded_at': ''})

funding_data_count = funding_data.groupby(['funded_at', 'State_Group'])['raised_amount_usd'].sum().reset_index(name='count')

In [49]:
# plot a lineplot over the aggregated amounts of funding, divided over the three stategroups
fig = px.line(funding_data_count, x = 'funded_at', y = 'count', color='State_Group', color_discrete_sequence = px.colors.sequential.YlGnBu[4:7])

# Update the layout, so headings etc. matches with the other visuals
fig.update_layout(
    title='Amount (in $) of fundings over years',
    font=dict(size=12),
    xaxis_title='Date of funding',
    yaxis_title='Funding in USD',
    legend_title_text='State Group',
    plot_bgcolor='white',
    font_family='Arial',
    title_x=0.5, 
    # Create the arrow and the text for the arrow
    annotations=[
        dict(
            x="2008-06-06", # define x-axis range
            y="50000000000", # define y-axis range 
            text="Fusion between Clearwire and Sprint Nextel",
            textangle=0,
            ax=200,
            ay=0,
            font=dict(
                color="black",
                size=12
            ),
            arrowcolor="black",
            arrowsize=2,
            arrowwidth=1,
            arrowhead=1),
]
)

fig.show()
pyo.plot(fig, filename='plot/funds.html')

'plot/funds.html'

There are some significant spikes we would now like to investigate also to give the reader some answers should they be left with question. The plot also shows even though some of the years were right after the financial crisis, a big investment was still made. This indicates that your idea may be more important than you timing, or that a least, the possibility of finding funding is always there even in times of crisis. 

#### Investigation of spikes

We now want to investigate the spike seen in 2009 for all three groups. We look at the companies and see if there is a correlation between the different areas of interest. 

In [50]:
# Create list with name of companies with 10 largest fundings in 2008-2009
funding89_data = funding_data[~(funding_data['funded_at'] < '2008-01-01')]
funding89_data = funding_data[~(funding_data['funded_at'] > '2009-12-31')]

#sort values by raised amount, so we get the largest fundings at the top
Company_Name = funding89_data.sort_values(by=['raised_amount_usd'], ascending=False)

# Drop duplicates if any, however included if e.g Facebook gets the two highest fundings and their different amounts. 
Company_Name = Company_Name.drop_duplicates(subset=['funded_object_id', 'raised_amount_usd'])

# Look at top 5
Company_Name = Company_Name['funded_object_id'][:5].array
Company_Name= objects[objects['id'].isin(Company_Name)][['name', 'category_code', 'State_Group']]
print("Companies within top 5 largest fundings in 2008-2009: \n", Company_Name)


# Create list with name of companies with 10 largest fundings in 2010-11
funding11_data = funding_data[~(funding_data['funded_at'] < '2010-01-01')] 
funding11_data = funding_data[~(funding_data['funded_at'] > '2011-12-31')]

Company_Name = funding11_data.sort_values(by=['raised_amount_usd'], ascending=False)
Company_Name = Company_Name.drop_duplicates(subset=['funded_object_id', 'raised_amount_usd'])
Company_Name = Company_Name['funded_object_id'][:5].array
Company_Name= objects[objects['id'].isin(Company_Name)][['name', 'category_code', 'State_Group']]
print("Companies within top 5 largest fundings in 2010-2011: \n", Company_Name)



# Create list with name of companies with 10 largest fundings in 2012-13
funding12_data = funding_data[~(funding_data['funded_at'] < '2012-01-01')]
funding12_data = funding_data[~(funding_data['funded_at'] > '2013-12-31')]

Company_Name = funding12_data.sort_values(by=['raised_amount_usd'], ascending=False)
Company_Name = Company_Name.drop_duplicates(subset=['funded_object_id', 'raised_amount_usd'])
Company_Name = Company_Name['funded_object_id'][:5].array
Company_Name= objects[objects['id'].isin(Company_Name)][['name', 'category_code', 'State_Group']]
print("Companies within top 5 largest fundings in 2012-2013: \n", Company_Name)

Companies within top 5 largest fundings in 2008-2009: 
                      name  category_code State_Group
4575            Clearwire         mobile      Others
81001            Solyndra  manufacturing          CA
170979  Fisker Automotive     automotive          CA
Companies within top 5 largest fundings in 2010-2011: 
              name category_code State_Group
2225      Groupon           web      Others
4575    Clearwire        mobile      Others
161551   Facebook        social          CA
Companies within top 5 largest fundings in 2012-2013: 
                                 name category_code State_Group
4575                       Clearwire        mobile      Others
97784                      sigmacare        health          NY
160023  Verizon Communications, Inc.        mobile          NY
161551                      Facebook        social          CA


Some of the companies that received a big amount of funding in the years with spikes is very well-known companies today. We see that we sorted the array, so we got the 5 largest fundings and then printed their name. However we also see that printed only shows respectively three and four companies. This means that some of the companies have more fundings in the top 5. 

We also looked further into Clearwire, as it appears in all spikes and found that they had a big merger in 2008, which explains the largest spike, the bot is not necessarily a learning that can be generally used. We now want to look into if there is anything else that can be more generally used, but looking at the timing of funding over months in a year and distributed on weekdays. 


### Investigation of Monthly and Weekly patterns of fundings

We want to look at the average data to make sure we catch the general trends. 

In [51]:
#Aggregation for weekday
funding_data_Weekday_agg = funding_data.groupby(['State_Group', 'Weekday'])['raised_amount_usd'].mean().reset_index(name='count')
funding_data_Weekday_agg = funding_data.groupby(['State_Group', 'Weekday'])['raised_amount_usd'].mean().unstack('State_Group').fillna(0) 
funding_data_Weekday_agg = funding_data_Weekday_agg.reset_index().rename_axis(columns={'Weekday': ''})

#Aggregation for Month
funding_data_Month_agg = funding_data.groupby(['State_Group', 'Month'])['raised_amount_usd'].mean().reset_index(name='count')
funding_data_Month_agg = funding_data.groupby(['State_Group', 'Month'])['raised_amount_usd'].mean().unstack('State_Group').fillna(0) 
funding_data_Month_agg = funding_data_Month_agg.reset_index().rename_axis(columns={'Month': ''})

In [52]:
# Define the desired order for the categories
# Months
order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
funding_data_Month_agg['Month'] = pd.Categorical(funding_data_Month_agg.Month, categories=order)
funding_data_Month_agg.sort_values(by='Month', inplace=True)

# Weekdays
order_Weekday = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
funding_data_Weekday_agg['Weekday'] = pd.Categorical(funding_data_Weekday_agg.Weekday, categories=order_Weekday)
funding_data_Weekday_agg.sort_values(by='Weekday', inplace=True)

In [53]:
# Make the interactive bar charts
plot = pg.Figure(data=[go.Bar(
    name='CA ',
    x=funding_data_Month_agg['Month'],
    y=funding_data_Month_agg['CA'],
    marker_color = '#7fcdbb'
),
    go.Bar(
    name='NY ',
    x=funding_data_Month_agg['Month'],
    y=funding_data_Month_agg['NY'],
    marker_color = '#2c7fb8'
),
    go.Bar(
    name='CA',
    x=funding_data_Weekday_agg['Weekday'],
    y=funding_data_Weekday_agg['CA'],
    marker_color = '#7fcdbb',
    visible=False
),
    go.Bar(
    name='NY',
    x=funding_data_Weekday_agg['Weekday'],
    y=funding_data_Weekday_agg['NY'],
    marker_color = '#2c7fb8',
    visible=False
)
])
  
  
# Add dropdown
plot.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="Monthly",
                     method="update",
                     args=[{"visible": [True, True, False, False]}, # Which of above Bar charts to visualize in the following dropdown menu
                           {"title": "Monthly",
                            }]),
                dict(label="Weekday",
                     method="update",
                     args=[{"visible": [False, False, True, True]}, # Which of above Bar charts to visualize in the following dropdown menu
                           {"title": "Weekday",
                            }]),
            ]),
        )
    ], 
    title='Average fundings by Month or Day of the Week',
    font=dict(size=12),
    yaxis_title='Average fundings', 
    plot_bgcolor='white',
    font_family='Arial',
    title_x=0.5)

  
plot.show()
pyo.plot(plot, filename='plot/month_week.html')

'plot/month_week.html'

There is no definite trend in either the monthly or weekday data, but there is some correlation with the financial year in terms of high average funding at the end of the second and third quarters and the beginning of the first. The plot is included to make sure the storyline is followed through, so the reader is not left with questions. 

## 3.4 Founders and Their Educational Background

We want to investigate to what degree, if any, the educational background of employees and founders has on startups. 

Below two bokeh plots are made over the most popular institutions sorted by the location of startups. As the focus so far has been on CA and NY, these two are still categorized for themselves and the rest is combined in a category 'other'. We want to know if people tend to move their startup to some of the know startups city after their education, or if the stay in the city where they have studied. Overall, we want to answer the question of whether startups are 'born' by cites, of if the move to the city. 

In [54]:
# Find the 15 institutions with the most employees
Top15_inst = founders[founders["institution"].isin(list(founders["institution"].value_counts().head(15).index))]

# Group by Institution and state_group in order to create the Bokeh plot. 
data_sub_count = Top15_inst.groupby(['institution', 'State_Group']).size().unstack('State_Group').fillna(0)

# Create a normalized dataframe
data_normalized = data_sub_count.div(data_sub_count.sum())

# Reset index and column names
data_normalized = data_normalized.reset_index().rename_axis(columns={'degree_type': ''})

In [55]:
# Name for Bokeh plot html-file for the website
output_file("plot/FoundersUni.html")

# Define state_groups for iterations
States = set(["NY", "CA", "Others"])

# Make list of all 15 institutions
inst = list(data_normalized['institution'])
insts = [str(i) for i in inst]

# Define colorscheme
colors = YlGnBu[len(States)]

# Create bar-chart, for count of founders per institution, divided on different state_groups
source = ColumnDataSource(data_normalized)
p = figure(title = 'Institutions where founders got their degree', x_range = FactorRange(factors=insts), x_axis_label="Institutions", y_axis_label="Normalized counts of founders per institution", min_width=900) 

bar ={} 
items = [] 
for indx,i in enumerate(States):
    bar[i] = p.vbar(x='institution',  top=i, source= source, 
                 muted_alpha=0.5, muted=False, color = colors[indx])
    items.append((i, [bar[i]])) 

legend = Legend(items=items, location='top_right') 
p.add_layout(legend, 'right') 

# makes sure to be able to interact
p.legend.click_policy="mute"

# makes legens vertical and readable
p.xaxis.major_label_orientation = np.pi/4
p.title = Title(text='Educational Backgrounds for Startups located in CA and NY and the Rest of US', align='center', text_font_size='12pt')

p.xaxis.axis_label_text_font_size = '12pt'
p.yaxis.axis_label_text_font_size = '12pt'
p.legend.label_text_font_size = '12pt'
show(p)

The plots show that education is widespread no matter where your startup is located, so it seems the educational institution is more connected to the nature of education, than where would be a good place for a startup. It is however seen that Standford is very popular, and after investigating this it was found that is known for having many graduates with startups. Now the question is what kind of degree they hold and we visualize that in the next plot. 

In [56]:
# get top 9 type of title within employees (e.g. Founder, CEO, accountant)
Top9 = founders[founders["title"].isin(list(founders["title"].value_counts().head(9).index))]

# get top 16 degree within top 9 emplyee title
Top9 = Top9[Top9["degree_type"].isin(list(Top9["degree_type"].value_counts().head(16).index))]

# Group above count basis their title and which degree they have
data_sub_count = Top9.groupby(['title', 'degree_type']).size().unstack('title').fillna(0)

# Create a normalized dataframe
data_normalized = data_sub_count.div(data_sub_count.sum())

# Reset index and column names
data_normalized = data_normalized.reset_index().rename_axis(columns={'degree_type': ''})

In [57]:
# Name for Bokeh plot html-file for the website
output_file("plot/e_level.html")

cds = ColumnDataSource(data_normalized)

# colors for Bokeh plot
colors = YlGnBu[9]

# create figure with degree on x-axis and count on y-axis. The legend consists of the different types of titles
p = figure(x_range=data_normalized["degree_type"], title="",
           x_axis_label="Degree", y_axis_label="Normalized Proportion", width=900, height=500)

bar = {}
items = [] 
for indx, i in enumerate(data_normalized.columns[1:]):
    bar[i] = p.vbar(x="degree_type", top=i, source=cds, fill_alpha=1, legend_label=i, width=0.6, color = colors[indx])
    items.append((i, [bar[i]])) 

# Layout changes, so the plot is more readable.
p.legend.click_policy = "hide"
p.legend.click_policy="mute"
p.add_layout(p.legend[0], 'right')
p.xaxis.major_label_orientation = np.pi/4
p.title = Title(text='Level of Education for People Working in Startups', align='center', text_font_size='12pt')

p.xaxis.axis_label_text_font_size = '12pt'
p.yaxis.axis_label_text_font_size = '12pt'
p.legend.label_text_font_size = '12pt'
show(p)

The plot shows a tendency for upper management to be educated in business and that software engineers have many different background. The overall story of the plot tells us that many different skills and knowledge sets can be needed to make a success of  a startup. 

### Startups categories and their employees educational area. 
We want to investigate further - is there a relationships between industry of business and the educational topic within the emplyees. As done ealier in the datacleaning, we need to merge some dataframes: 

In [58]:
founder_education = pd.merge(founders ,main_data[['id_x', 'category_code']], left_on='relationship_object_id', right_on='id_x', how='left')

Here is especially the category of the `subject` of the degree and the `category_code` of the company important, we thereby delete those null-values.

In [59]:
founder_education.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159461 entries, 0 to 159460
Data columns (total 35 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id_x_x                  159461 non-null  int64         
 1   id_x_x                  159461 non-null  float64       
 2   relationship_id         159461 non-null  int64         
 3   person_object_id        159461 non-null  object        
 4   relationship_object_id  159461 non-null  object        
 5   title                   155687 non-null  object        
 6   id_y                    159411 non-null  float64       
 7   id_y                    159461 non-null  float64       
 8   object_id_x             159411 non-null  object        
 9   first_name              159411 non-null  object        
 10  last_name               159411 non-null  object        
 11  birthplace              19202 non-null   object        
 12  affiliation_name        159392

In [60]:
founder_education.dropna(subset=['subject', 'category_code'], inplace=True)

As we see a big change in counts after advertising, we only choose to look at top 9. This means we include two other areas of industries after the change in counts.

In [61]:
founder_education["category_code"].value_counts().head(15)

software           15180
search             12738
web                 8341
enterprise          6256
biotech             5889
consulting          5229
advertising         5091
mobile              3853
other               3347
hardware            2933
ecommerce           2630
games_video         2550
cleantech           1913
network_hosting     1826
security            1535
Name: category_code, dtype: int64

In [62]:
list = founder_education["category_code"].value_counts().head(9).index

# get top 9 type of title within employees (e.g. Founder, CEO, accountant)
Top20 = founder_education[founder_education["category_code"].isin(list)]

# get top 16 degree within top 9 emplyee title
Top20 = Top20[Top20["subject"].isin(Top20["subject"].value_counts().head(10).index)]

# Group above count basis their title and which degree they have
data_sub_count = Top20.groupby(['category_code', 'subject']).size().unstack('category_code').fillna(0)

# Reset index and column names
data_sub_count = data_sub_count.reset_index().rename_axis(columns={'subject': ''})


In [63]:
data_sub_count

category_code,subject,advertising,biotech,consulting,enterprise,mobile,other,search,software,web
0,Business,100,69,120,165,76,72,223,320,200
1,Business Administration,120,132,193,148,95,102,259,411,171
2,Computer Engineering,34,8,33,107,75,32,268,243,114
3,Computer Science,409,69,239,744,406,184,2539,1901,751
4,Economics,268,162,292,245,174,147,412,607,321
5,Electrical Engineering,128,114,185,296,205,133,451,737,235
6,Finance,142,118,157,166,100,108,174,433,257
7,Law,104,95,85,88,56,69,115,170,145
8,Marketing,133,58,61,128,87,41,183,252,144
9,Mechanical Engineering,37,140,158,129,52,83,179,244,93


In [64]:
# Name for Bokeh plot html-file for the website
output_file("plot/e_level_category.html")

cds = ColumnDataSource(data_sub_count)

# colors for Bokeh plot
colors = YlGnBu[9]

# create figure with degree on x-axis and count on y-axis. The legend consists of the different types of titles
p = figure(x_range=data_sub_count["subject"], title="",
           x_axis_label="Subject in education", y_axis_label="Count of subjects", width=900, height=500)

bar = {}
items = [] 
for indx, i in enumerate(data_sub_count.columns[1:]):
    bar[i] = p.vbar(x="subject", top=i, source=cds, fill_alpha=1, legend_label=i, width=0.6, color = colors[indx])
    items.append((i, [bar[i]])) 

# Layout changes, so the plot is more readable.
p.legend.click_policy = "hide"
p.legend.click_policy="mute"
p.add_layout(p.legend[0], 'right')
p.xaxis.major_label_orientation = np.pi/4
p.title = Title(text='Allocation of subjects within different types of industries', align='center', text_font_size='12pt')

p.xaxis.axis_label_text_font_size = '12pt'
p.yaxis.axis_label_text_font_size = '12pt'
p.legend.label_text_font_size = '12pt'

show(p)

# 4. Genre

### Data story genre 

The article follows an Annotated chart structure with interactive visualization. The story is mostly on the author-driven side of the spectrum, except there is an allowance for interactivity with certain visualization. This provides the users with the opportunity to explore aspects, not necessarily mentioned in the article. 




### Visual Narrative Tools

Each narrative tool used in the article is explained below. 

**Consistent visual platform** 

* The layout has been carefully thought out, following the same color scheme and making sure the backgrounds on the plots are the same as on the website for a homogeneous impression. All color and stylish choices have been made with the goal of consistency and easy readability. The sidebar with the header is stationary reminding the reader of the angle the data is perceived throughout the whole storyline. 

**Feature distinction** 

* The data is often categorized, proving the reader with more perspectives and comparing our findings across locations. An example is the plot over the level of education for people working in startups. The feature of positions helps analyze the connection between education and the makeup of startups.  

**Zooming** 

* Many of the plots especially the maps provide the ability to zoom, to provide more detail for the curious reader. 






### Narrative Structure Tools

Each narrative structure tool used in the article is explained below. 

**Hover Highlighting / details** 

* We have tried to make the plots as interactive as possible, so you can hover over most of the bar plots that are not bokhe and the heat maps to get additional information. 

**Filtering /selection /search**  

* A filter or selection option is included in all bokeh plots, which is a big part of the reason the particular plot type was chosen. It enables the reader to easly compare subjects that interest the reader the most. 

**Captions /headlines** 

* The storyline and reader direction is guided by headlines and figure captions. This ensures a red thread through the finding in our data. The headlines are the key to a structured and clean article. 

**Introductory text**

* The introductory text is always made for a new section, as there may also be a need for additional contextual information not found only in the data, but also found in general knowledge about businesses and cities. 

**Summary** 

* Summaries of all findings from our plot are included as part of the storyline. 

**Annotation** 

* One of the plots uses annotation, as there is a big unexplained spike, that we did not believe could be left explained only by the summary of the finding below. Instead, it is included in the plot enabling the reader to get an overview of the most important takeaways at first glance. 

# 6. Discussion

***What went weel?***

The visualization used in the article ended up being intuitive and interactive and the overall storyline provides the reader with new information. Furthermore, we believe, there are findings in the article that would be surprising for our readers, maybe especially the ones residing outside the US. For example, the fact that several employees from startups have graduated from Standford. Overall we think that we ended up telling a good story about how startups influence their environment and how important it is to place a startup in a big city. 

***What is still missing and what could be improved?***

The fact that the datasets did not contain a lot of closed startups ended up being a disadvantage. It has made it hard to asses what makes a successful startup, as it is hard to define success without having the opportunity to compare it with failure. Furthermore, looking into startups from a data perspective leaves a lot of gaps to be filled out, as it has been discovered that the startups are of complex size. Besides measurable values such as funding and several offices, startups are also highly based on the drive and motivation of the people involved, and sometimes it is seer preserve that keeps a startup running. This is hard to capture in data. We tried to look into the sentiment of the milestones, to capture the tendencies of some of these more soft values, but unfortunately, it did not yield any useful results and was removed again. 

Lastly, our data has been more suitable for analysis on a state level, which has made it harder to make valuable analyses on a city level. For further work on this article, it could be advantageous to look into more general city data and see if there could be some connection to our startup data, but again it is something that would provide more value if we had a clearer definition of what a successful startup is made out of.

# 7. Contributions

In [65]:
contribution  = {
    'Helene Hjort (194665)': ['40%', '20%', '35%', '30%', '50%', '25%'],
    'Kathrine Sofie Rasmussen (s194654)': ['30%', '40%', '35%', '50%', '25%', '20%'],
    'Sascha Thorsgaard Jacobsen (s171281)': ['30%', '40%', '30%', '20%', '25%', '55%']
}

index = ['Data cleaning', 'Descriptive analyse', 'Analyse', 'Visualization', 'Website', 'Text']

contribution = pd.DataFrame(contribution, index=index)
contribution


,Helene Hjort (194665),Kathrine Sofie Rasmussen (s194654),Sascha Thorsgaard Jacobsen (s171281)
Data cleaning,40%,30%,30%
Descriptive analyse,20%,40%,40%
Analyse,35%,35%,30%
Visualization,30%,50%,20%
Website,50%,25%,25%
Text,25%,20%,55%
